<a href="https://colab.research.google.com/github/mag381/dog/blob/main/Efficient_AD_Segmentation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

異常検知】学習ゼロの衝撃！を可視化する by @shinmura0
https://qiita.com/shinmura0/items/5f2c363812f7cdcc8771
Modeling the Distribution of Normal Data in Pre-Trained Deep Features for Anomaly Detection
https://arxiv.org/abs/2005.14140

In [6]:
####### input condition ############
model_name = 'tf_efficientnet_b6_ns'
level = 7

#target = "capsule"
#target = "screw"
#target = "pill"
#target = "bottle"
#target = "grid"
#target = "zipper"
#target = "transistor"
#target = "wood"
#target = "toothbrush"
#target = "tile"
#target = "metal_nut"
#target = "hazelnut"
target = "cable"
#target = "leather"
#target = "carpet"

RESIZE = 528
sample_no = 54
####################################

In [ ]:
pip install timm

In [ ]:
to_color_list = ["screw", "grid", "zipper"]
to_color = False

if target in to_color_list:
    to_color = True
print("to_color is ",to_color)

device="cuda"

In [4]:
from PIL import Image
import glob
import sys, os, urllib.request, tarfile, cv2
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as data
from typing import Optional, List
from torchvision import models, transforms
import random
import torch
from tqdm.notebook import tqdm

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
def MVTechAD(download_dir, path):
    target_path = "data/"

    if not os.path.exists(download_dir):
        os.mkdir(download_dir)

    # download file
    def _progress(count, block_size, total_size):
        sys.stdout.write('\rDownloading %s %.2f%%' % (source_path,
            float(count * block_size) / float(total_size) * 100.0))
        sys.stdout.flush()

    source_path = path
    dest_path = os.path.join(download_dir, "data.tar.xz")
    urllib.request.urlretrieve(source_path, filename=dest_path, reporthook=_progress)
    # untar
    with tarfile.open(dest_path, "r:xz") as tar:
        tar.extractall(target_path)
class ImageTransform():
    def __init__(self, resize=RESIZE):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                #transforms.RandomCrop(224),
                #transforms.RandomRotation(20, fill=200),
                #transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # 0-1 → 標準化
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                #transforms.CenterCrop(224),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # 0-1 → 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)
def fig_show(img):
    # 2. 元の画像の表示
    plt.imshow(img)
    plt.title("Original")
    plt.show()

    transform = ImageTransform()
    img_transformed = transform(img, phase="train")  # torch.Size([3, 224, 224])

    # (色、高さ、幅)を (高さ、幅、色)に変換し、0-1に値を制限して表示
    plt.subplot(1,2,1)
    img_transformed = img_transformed.numpy().transpose((1, 2, 0))
    img_transformed = np.clip(img_transformed, 0, 1)
    plt.imshow(img_transformed*255)
    plt.title("Train")
    plt.show()

set_seed(1213)

In [ ]:
if not os.path.isdir("data/" + target):
    MVTechAD("./ad", "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937413-1629951498/" + target + ".tar.xz")

image_file_path = 'data/' + target + '/train/good/000.png'
img = Image.open(image_file_path).convert("RGB")   # [高さ][幅][色RGB]

fig_show(img)   

In [ ]:
from sklearn.model_selection import train_test_split

train_list = glob.glob("data/" + target + "/train/**/**.png")
test_list = glob.glob("data/" + target + "/test/**/**.png")

normal_list, anomaly_list = [], []

for i in range(len(test_list)):
    if "good" in test_list[i]:
        normal_list.append(test_list[i])
    else:
        anomaly_list.append(test_list[i])

print(len(train_list))
print(len(normal_list))
print(len(anomaly_list))

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, 
                 train_list: List[List[str]], 
                 to_color="False"):
        self.file_list = train_list
        self.to_color = to_color
        self.transform = ImageTransform()

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx: int):
        img_path = self.file_list[idx]
        if self.to_color:
            img = Image.open(img_path).convert("RGB")  # [高さ][幅][色RGB]
        else:
            img = Image.open(img_path)

        # 画像の前処理を実施
        img = self.transform(img, "train")

        return img

# 実行
train_dataset = Dataset(train_list, to_color=to_color)
normal_dataset = Dataset(normal_list, to_color=to_color)
anomaly_dataset = Dataset(anomaly_list, to_color=to_color)

# 動作確認
index = 25
output = anomaly_dataset.__getitem__(index)
print(output.size())

Make model

In [ ]:
import timm
from timm.models.efficientnet import EfficientNet
import torch
import torch.nn.functional as F

def extract_features(inputs: torch.Tensor, 
                     model: EfficientNet,
                     level):
    features = dict()
    # extract stem features as level 1
    x = model.conv_stem(inputs)
    x = model.bn1(x)
    x = model.act1(x)
    features['level_1'] = F.adaptive_avg_pool2d(x, 1)
    # extract blocks features as level 2~8
    for i, block_layer in enumerate(model.blocks):
        x = block_layer(x)
        features[f'level_{i+2}'] = F.adaptive_avg_pool2d(x, 1)
    # extract top features as level
    x = model.conv_head(x)
    x = model.bn2(x)
    x = model.act2(x)
    features['level_9'] = F.adaptive_avg_pool2d(x, 1)
    return features['level_{}'.format(str(level))]

model = timm.create_model(model_name, pretrained=True)
model.eval()
model.to(device)

Extract feature

In [ ]:
import torch
from sklearn.covariance import LedoitWolf

def get_mean_cov(loader):
    feat = []

    for inputs in loader:
        inputs = inputs.to(device)
        feat_list = extract_features(inputs, model, level) # levelは1~9のint, featuresは上述のextract_features()結果
        feat_list = feat_list.cpu().detach().numpy()
        #print(feat_list.shape)
        for i in range(len(feat_list)):
            feat.append(feat_list[i].reshape(-1))

    feat = np.array(feat)
    #print(feat.shape)

    mean = np.mean(feat, axis=0)
    cov = np.cov(feat.T)

    return feat, mean, cov

train_loader = data.DataLoader(train_dataset, 
                               batch_size=1, 
                               shuffle=False, 
                               num_workers=2, 
                               pin_memory=True, 
                               drop_last=True)

normal_loader = data.DataLoader(normal_dataset, 
                                batch_size=1, 
                                shuffle=False, 
                                num_workers=2, 
                                pin_memory=True, 
                                drop_last=True)

anomaly_loader = data.DataLoader(anomaly_dataset, 
                                batch_size=1, 
                                shuffle=False, 
                                num_workers=2, 
                                pin_memory=True, 
                                drop_last=True)

train_feat, mean, cov = get_mean_cov(train_loader)
normal_feat, _, _ = get_mean_cov(normal_loader)
anomaly_feat, _, _ = get_mean_cov(anomaly_loader)

print(mean.shape, cov.shape)

In [13]:
from scipy.spatial import distance

def get_score(feat, mean, cov):
    result = []
    # 分散共分散行列の逆行列を計算
    cov_i = np.linalg.pinv(cov)

    for i in range(len(feat)):
        result.append(distance.mahalanobis(feat[i], mean, cov_i))
    return result, cov_i

normal_score, cov_i = get_score(normal_feat, mean, cov)
anomaly_score, _ = get_score(anomaly_feat, mean, cov)

Detection AUC

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics

def get_auc(Z1, Z2):
    plt.title("Mahalanobis distance")
    plt.plot(Z1, label="normal")
    plt.plot(Z2, label="anomaly")
    plt.legend()
    plt.show()

    y_true = np.zeros(len(Z1)+len(Z2))
    y_true[len(Z1):] = 1#0:正常、1：異常

    # FPR, TPR(, しきい値) を算出
    fpr, tpr, _ = metrics.roc_curve(y_true, np.hstack((Z1, Z2)))

    # AUC
    auc = metrics.auc(fpr, tpr)

    return fpr, tpr, auc

def plot_roc(Z1, Z2, name):
    fpr, tpr, auc1 = get_auc(Z1, Z2)
    plt.plot(fpr, tpr, label=name + '(AUC = %.3f)'%(auc1))

    plt.legend()
    plt.title(name + '(ROC)')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

    return auc1

_ = plot_roc(normal_score, anomaly_score, target)

Visualization

In [15]:
def get_score_pytorch(feature, mean, cov_i):
    mean = torch.from_numpy(mean).to(device).float()
    cov_i = torch.from_numpy(cov_i).to(device).float()
    result = torch.matmul(feature - mean, cov_i)
    result = torch.matmul(result, feature - mean)
    print("Mahalanobis distance", torch.sqrt(result))
    return torch.sqrt(result)

def make_fig(fig):
    img_mean = [0.485, 0.456, 0.406]
    img_std = [0.229, 0.224, 0.225]
    img = fig.clone().cpu().detach().numpy()
    img = img[0].transpose(1,2,0)
    for i in range(3):
        img[:,:,i] = img[:,:,i]*img_std[i]+img_mean[i]
    return img

In [ ]:
# ハイパラ
alpha = 0.0001
lamda = 1

figures=[]

# get figure
no = 0
for x_0 in anomaly_loader:
    if no==sample_no:
        break
    no+=1

figures.append(make_fig(x_0))

# initial backward
x_0 = x_0.to(device).clone().detach().requires_grad_(True)
feature = extract_features(x_0.to(device), model, level) # levelは1~9のint, featuresは上述のextract_features()結果
loss = get_score_pytorch(feature.view(-1), mean, cov_i)
loss.backward(retain_graph=True)

x_grad = x_0.grad.data
x_t = x_0 - alpha * x_grad

# backward iteration
for i in range(50):
    feature = extract_features(x_t.to(device), model, level) # levelは1~9のint, featuresは上述のextract_features()結果
    score = get_score_pytorch(feature.view(-1), mean, cov_i)
    loss = score + lamda * torch.abs(x_t - x_0).sum()
    loss.backward(retain_graph=True)

    x_grad = x_0.grad.data
    x_t = x_t - alpha * x_grad
    figures.append(make_fig(x_t))

Result

In [ ]:
import cv2

diff = np.abs(figures[0]-figures[-1])
diff = np.sum(diff, axis=-1)
diff = (diff-np.min(diff))/(np.max(diff)-np.min(diff))

#plt.imshow(diff, cmap="jet")#figures[0]-figures[-1])
jetcam = cv2.applyColorMap(np.uint8(255 * diff), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
jetcam = (np.float32(jetcam) + figures[0] / 2)   # もとの画像に合成

plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.axis("off")
plt.imshow(figures[0])

plt.subplot(1,2,2)
plt.imshow(jetcam)
plt.axis("off")
plt.show()

References
https://qiita.com/kogepan102/items/122b2862ad5a51180656
https://qiita.com/sUeharaE4/items/023455822376c07cd6ef
https://www.mvtec.com/company/research/datasets/mvtec-ad